In [ ]:
from loica import *
import matplotlib.pyplot as plt

In [ ]:
osc = GeneticNetwork()

acts = [Regulator(name=f'Act{i}', degradation_rate=0.75) for i in range(3)]
osc.add_regulators(acts)

rep = Regulator(name='Rep', degradation_rate=0.75)
osc.add_regulator(rep)

reporters = [Reporter(name=f'SFP{i}', color='blue', degradation_rate=0.75) for i in range(3)]
osc.add_reporters(reporters)

op1 = Hill2([rep,acts[0]], [acts[0],acts[1],reporters[0]], alpha=[100,0,10000,0], K=[1000,1000], n=[2,2])
op2 = Hill1(acts[1], [acts[2], reporters[1]], alpha=[100,10000], K=1000, n=2)
op3 = Hill1(acts[2], [rep, reporters[2]], alpha=[100,10000], K=1000, n=2)
osc.add_operators([op1,op2,op3])

In [ ]:
plt.figure(figsize=(4,4), dpi=150)
osc.draw(contracted=False, arrowsize=7, node_size=500, linewidths=0, alpha=1)

In [ ]:
def growth_rate(t):
    return gompertz_growth_rate(t, 0.01, 1, 1, 0.5)

def biomass(t):
    return gompertz(t, 0.01, 1, 1, 0.5)

metab = SimulatedMetabolism(biomass, growth_rate)

In [ ]:
acts[0].init_concentration = 100

In [ ]:
sample = Sample(genetic_network=osc, 
                    metabolism=metab)

In [ ]:
assay = Assay([sample], 
              n_measurements=200, 
              interval=0.24,
              name=f'Loica Tigges oscillator',
              description='Simulated oscillator generated by loica'
             )
assay.run()

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
for i in range(3):
    m[m.Signal==f'SFP{i}'].plot(x='Time', y='Measurement', style='-', ax=ax)
plt.legend([f'SFP{i}' for i in range(3)])

In [ ]:
col = Colony(circuit=osc, r0=10)

In [ ]:
kymo = col.map_kymo(col.kymograph(500, 0, 24))

In [ ]:
kymo.shape

In [ ]:
plt.imshow(kymo)